In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [18]:
df = pd.read_csv('input/Synthetic_data_grid_path_1.txt',delimiter=',',encoding='latin-1',header=-1)
df.head()

,0
0,W3 S5 SE6 S8 N10
1,N3 SE5 S6 SE7 E8
2,N4 W5 S6 NW7 W8 S10
3,N4 NE5 E10
4,S2 W3 SW5


In [19]:
df[1]=df[0].str[-3:]
df[0]=df[0].str[:-3]

In [20]:
df[1]=df[1].str[:2]

In [21]:
df[1]=df[1].str.strip()

In [22]:
df[1]=df[1].str[:1]

In [23]:
df[1]

0       N
1       E
2       S
3       E
4       S
5       S
6       S
7       N
8       N
9       W
10      N
11      E
12      S
13      N
14      E
15      W
16      E
17      W
18      S
19      S
20      S
21      N
22      S
23      N
24      S
25      W
26      S
27      N
28      N
29      N
       ..
9902    S
9903    S
9904    E
9905    W
9906    S
9907    N
9908    N
9909    W
9910    W
9911    W
9912    N
9913    W
9914    W
9915    E
9916    S
9917    S
9918    N
9919    E
9920    S
9921    E
9922    W
9923    S
9924    S
9925    E
9926    W
9927    S
9928    S
9929    N
9930    W
9931    S
Name: 1, Length: 9932, dtype: object

In [26]:
X = df[0]
Y = df[1]
le = LabelBinarizer()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,4)

In [27]:
Y

array([[0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       ...,
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]])

In [28]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [29]:
Y_train

array([[0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1]])

In [30]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [31]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(4,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [32]:
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 4)                 1028

In [33]:
model.fit(sequences_matrix,Y_train,batch_size=1,epochs=10,
          validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.001)])

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7597 samples, validate on 845 samples
Epoch 1/10
7597/7597 [==============================] - 169s 22ms/step - loss: 1.3607 - accuracy: 0.2796 - val_loss: 1.3857 - val_accuracy: 0.2923
Epoch 2/10
7597/7597 [==============================] - 167s 22ms/step - loss: 1.3595 - accuracy: 0.2792 - val_loss: 1.4295 - val_accuracy: 0.2828


In [34]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [35]:
accr = model.evaluate(test_sequences_matrix,Y_test)

1490/1490 [==============================] - 1s 387us/step


In [36]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 1.411
  Accuracy: 0.292
